In [1]:
pip install lightgbm


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 📂 Charger le dataset des films
df = pd.read_csv("movie_data_cleaned.csv")

# 🔄 Sélection des colonnes pertinentes
features = ["popularity", "vote_average", "runtime", "genres"]
df = df[["title"] + features]

# 🔄 Encodage des genres en valeurs numériques
label_encoder = LabelEncoder()
df["genres_encoded"] = label_encoder.fit_transform(df["genres"])

# 💾 Sauvegarde des données préparées
df.to_csv("movie_data_prepared.csv", index=False)
print("✅ Données prêtes pour l'entraînement !")


✅ Données prêtes pour l'entraînement !


In [9]:
import lightgbm as lgb
import joblib

# 📂 Charger les données préparées
df = pd.read_csv("movie_data_prepared.csv")

X = df[["popularity", "vote_average", "runtime", "genres_encoded"]]
y = np.random.rand(len(X))  # Simuler une cible (à remplacer par des vraies données)

# 🔄 Diviser en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🚀 Entraîner LightGBM
model = lgb.LGBMRegressor(n_estimators=300, learning_rate=0.05, max_depth=5, num_leaves=50)
model.fit(X_train, y_train)

# 💾 Sauvegarder le modèle
joblib.dump(model, "lightgbm_model.pkl")
print("✅ Modèle LightGBM entraîné et sauvegardé avec succès !")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000792 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 842
[LightGBM] [Info] Number of data points in the train set: 23764, number of used features: 4
[LightGBM] [Info] Start training from score 0.501693
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [10]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics.pairwise import cosine_similarity

# 📂 Charger le modèle et les données
movies_df = pd.read_csv("movie_data_prepared.csv")
model = joblib.load("lightgbm_model.pkl")

def recommend_movies(movie_title, k=5):
    """Retourne k films similaires au film donné par l'utilisateur"""
    
    if movie_title not in movies_df["title"].values:
        return "❌ Film non trouvé."

    # 🔍 Récupérer les caractéristiques du film
    movie_features = movies_df[movies_df["title"] == movie_title][["popularity", "vote_average", "runtime", "genres_encoded"]]

    # 🔥 Trouver les films similaires avec LightGBM
    movies_df["score"] = model.predict(movies_df[["popularity", "vote_average", "runtime", "genres_encoded"]])

    # 🎬 Sélection des meilleurs films
    recommendations = movies_df.sort_values("score", ascending=False).head(k)[["title", "score"]]
    
    return recommendations

# 🎬 Exemple d'utilisation
user_movie = input("🎬 Entrez un titre de film : ")
print(recommend_movies(user_movie))


                        title     score
20597       Story of My Death  0.740905
999    The Godfather: Part II  0.731354
26313     Someone to Run With  0.698305
697             The Godfather  0.695384
27036              Your Name.  0.690457
